### Paul Spendlove
### A02242861

# Data Generation Scripts

In [1]:
# %pip install yfinance
import requests
import yfinance as yf
import pandas as pd

### 1.

Load all NASDAQ tickers into columns, with price traded for each date in stock market history as the rows.

In [2]:
nasdaq_traded_url = "http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt"
response = requests.get(nasdaq_traded_url)
data = response.content.decode('utf-8')

Run this to create the big dataset.  

Estimated time running: 10 - 15 minutes (sorry)

In [3]:
symbols = [line.split('|')[1] for line in data.splitlines()[1:] if len(line.split('|')) >= 8 if line.split('|')[7] == 'N' and line.split('|')[5] == 'N']

symbols = list(set(symbols))

data2 = yf.download(symbols, period="max")["Close"]

data2.to_csv("nasdaq_historical_data.csv")

[*********************100%%**********************]  7584 of 7584 completed

619 Failed downloads:
['GPMT$A', 'AXS$E', 'PSA$Q', 'MDV$A', 'VNO$L', 'GMRE$A', 'PL.W', 'RIV$A', 'GSL$B', 'DSX$B', 'F$D', 'EFC$B', 'O$', 'REXR$C', 'JPM$L', 'CMS$B', 'SEAL$A', 'PCG$A', 'JOBY.W', 'RMPL$', 'PEB$F', 'SCHW$D', 'SB$C', 'IIPR$A', 'SCE$L', 'GLU$A', 'USB$P', 'USB$H', 'EBR.B', 'GS$C', 'PBR.A', 'CNO$A', 'BNRE.A', 'WBS$G', 'LEGT.U', 'TRIS.W', 'HIG$G', 'PCG$B', 'GDV$H', 'PCG$G', 'ALB$A', 'CIM$B', 'WFC$L', 'BMTX.W', 'HVT.A', 'BFS$D', 'AP.W', 'PSA$S', 'MET$A', 'AHT$H', 'CLVT$A', 'CFR$B', 'NGL$C', 'RDW.W', 'EQC$D', 'DMYY.W', 'DLR$J', 'FHN$E', 'MS$O', 'CVE.W', 'MKFG.W', 'PHYT.U', 'HL$B', 'ARGO$A', 'SBXC.W', 'SPNT$B', 'BAC$N', 'PRIF$K', 'SHO$I', 'GAB$H', 'NYCB$A', 'PRIF$H', 'ETWO.W', 'BFLY.W', 'NCV$A', 'OXY.W', 'KIM$L', 'PSA$I', 'CMRE$B', 'C$N', 'EFC$A', 'RJF$B', 'BC$A', 'COF$J', 'ATH$A', 'ACR$D', 'AHL$E', 'SMR.W', 'PCG$C', 'JWSM.W', 'OAK$A', 'BML$H', 'WSO.B', 'PSA$R', 'SYF$B', 'NEE$R', 'CMS$C', 'AHT$I', 'GS$K', 

### 2.

Create new file mapping each stock ticker to a readable stock name.

Estimated time running: 1 minute

In [4]:
ticker_info = pd.DataFrame(columns=['Ticker', 'Company Name'])

for line in data.splitlines()[1:]:
    if len(line.split('|')) >= 8:
        if line.split('|')[7] == 'N' and line.split('|')[5] == 'N':
            ticker_info = ticker_info.append({'Ticker': line.split('|')[1], 'Company Name': line.split('|')[2]}, ignore_index=True)


ticker_info.to_csv("ticker_company_name_map.csv", index=False)

### 3. 

Load data for the top 15 stocks with the highest market cap (roughly equivalent to the top few of the S&P 500)

In [5]:
market_cap_data = pd.read_csv('nasdaq_screener.csv')

top_market_caps = market_cap_data.sort_values(by='Market Cap', ascending=False).head(15)

top_15_symbols = top_market_caps['Symbol'].tolist()

high_cap_nasdaq_symbols = [symbol for symbol in top_15_symbols]

data_high_cap = yf.download(high_cap_nasdaq_symbols, period="max")["Close"]

data_high_cap.to_csv("nasdaq_high_market_cap_historical_data.csv")

[**********************60%%***                   ]  9 of 15 completedFailed to get ticker 'BRK/B' reason: Expecting value: line 1 column 1 (char 0)
[**********************67%%******                ]  10 of 15 completedFailed to get ticker 'BRK/A' reason: Expecting value: line 1 column 1 (char 0)
[*********************100%%**********************]  15 of 15 completed

2 Failed downloads:
['BRK/B', 'BRK/A']: Exception('%ticker%: No timezone found, symbol may be delisted')
